In [1]:
# Import cell

import xarray as xr # to work with multi-dimensional arrays
import numpy as np # to work with multi-dimensional arrays
import regionmask # to work with predefined data masks
import pandas as pd # to work with tables
import xesmf as xe # to regrid data
import glob # to find file pathways
import netCDF4 # use to save xarrays as nc files
import matplotlib
import matplotlib.pyplot as plt # for plotting
import cartopy.crs as ccrs # use for geographic map projections
import time # to add a creation timestamp to Datasets
import pickle # to save DataFrame objects efficiently

### Spatial Mean and the Standard Deviation of Absolute Annual Near Surface Temperature: All Models

Raw monthly near surface temperature data has been accessed from the CEDA archives via JASMIN resources, and will be used to determine spatial mean annual near surface temperature and its associated standard deviation for a range of CMIP6 models.

Metacode:

1. Restrict Dataset/DataArray object to specified time period. (e.g. 2071-2100.)
2. Compute the annual near surface temperature for each year within the period by averging the monthly values.
3. Compute the mean annual near surface temperature and the standard deviation of these means across the period.
4. Form a Dataset object with these two variables.
5. Save Datatset object.

As essentially only working out a time average across cells, no need for weighting to account for change in cell area with latitude.

In [2]:
def ann_tas(arr, start_year, end_year):
    
    '''
    Summary:
    --------
    Computes the spatial mean and standard deviation of annual surface temperature from monthly data
    across a specified range.
    
    Paramters:
    ----------
    arr: xarray dataset
         xarray object must be contain monthly interval with surface temperature variable (tas)
         xarray object must be chronologically ordered
         
    start_year: str
                starting year of the temporal range to be used
    
    end_year: str
              end year of the temporal range to be used
    
    Returns:
    --------
    ann_tas: xarray dataset
                  
    '''
    # load air surface temperature variable in
    tas_arr = arr.tas

    # select only values between specified start and end years
    tas_arr = tas_arr.sel(time = slice(start_year, end_year, 1)) # use slice to index the time dimension

    # convert temperature units from Kelvin to degrees Celcius
    tas_arr = tas_arr - 273.15

    # compute the yearly spatial mean temperature for each year; average monthly temperatures for given year
    tas_arr = tas_arr.groupby('time.year').mean(skipna = True)

    # compute the spatial mean annual temperature across the specified period
    mean_tas = tas_arr.mean(dim = 'year', skipna = True)

    # compute the spatial standard deviation of the mean annual temperature across the specified period
    std_dev_tas = tas_arr.std(dim = 'year', skipna = True)

    # rename the DataArray objects
    mean_tas = mean_tas.rename('mean_ann_tas')
    std_dev_tas = std_dev_tas.rename('std_dev_ann_tas')

    # define some attributes for the mean and standard deviation DataArray objects
    mean_tas.attrs['units'], std_dev_tas.attrs['units'] = 'degrees Celcius', 'degrees Celcius'
    mean_tas.attrs['sample_start_year'], std_dev_tas.attrs['sample_start_year'] = start_year, start_year
    mean_tas.attrs['sample_end_year'], std_dev_tas.attrs['sample_end_year'] = end_year, end_year
    mean_tas.attrs['model'], std_dev_tas.attrs['model'] = arr.source_id, arr.source_id
    mean_tas.attrs['modelling_group'], std_dev_tas.attrs['modelling_group'] = arr.institution, arr.institution
    mean_tas.attrs['modelling_group_id'], std_dev_tas.attrs['modelling_group_id'] = arr.institution_id, \
                                                                                    arr.institution_id
    mean_tas.attrs['experiment'], std_dev_tas.attrs['experiment'] = arr.experiment_id, arr.experiment_id
    mean_tas.attrs['realisation'], std_dev_tas.attrs['realisation'] = arr.variant_label, arr.variant_label

    # create Dataset object from the mean and standard deviation DataArray objects
    ann_tas = xr.Dataset({'mean_tas': mean_tas, 'std_dev_tas': std_dev_tas})

    # define some attributes for the Dataset object
    ann_tas.attrs['comment'] = 'spatial annual mean and standard deviation of near surface temperature'
    ann_tas.attrs['units'] = 'degrees Celcius'
    ann_tas.attrs['sample_start_year'] = start_year
    ann_tas.attrs['sample_end_year'] = end_year
    ann_tas.attrs['model'] = arr.source_id
    ann_tas.attrs['modelling_group'] = arr.institution
    ann_tas.attrs['modelling_group_id'] = arr.institution_id
    ann_tas.attrs['experiment'] = arr.experiment_id
    ann_tas.attrs['realisation'] = arr.variant_label
    ann_tas.attrs['created'] = time.ctime()
    
    # close open Dataset and/or DataArray objects
    mean_tas.close(), std_dev_tas.close(), tas_arr.close(), arr.close()
    
    return ann_tas

In [3]:
def merge_arrays_via_time(pw_list):
    
    '''
    Summary:
    --------
    Merges xarrays with the same attributes along the time dimension.
    
    Parameters:
    -----------
    pw_list: list
             list containing the absolute pathways of the xarrays to be merged
             
    Returns:
    --------
    merged_array: xarray
                  chronologically ordered xarray containing all arrays listed in pw_list
    
    '''
    # load first array
    x = xr.open_dataset(pw_list[0], use_cftime = True) # cftime is a package that decodes netCDF files times
                                                              # times to conform with climate and forecasting times
    merged_array = x
    x.close()
    del(x)

    # merge remaining arrays
    for pw in pw_list[1: ]:
        x = xr.open_dataset(pw, use_cftime = True).load()
        merged_array = xr.concat([merged_array, x], dim = 'time')
        x.close()
        del(x)
        
    # sort resultant array by time; earliest to latest
    merged_array = merged_array.sortby('time')

    return merged_array

In [17]:
# define a list containing the CMIP6 models used in study
models = ['ACCESS-CM2', 'ACCESS-ESM1-5', 'AWI-CM-1-1-MR', 'BCC-CSM2-MR', 'CanESM5',
          'CNRM-CM6-1', 'GFDL-ESM4', 'INM-CM4-8', 'INM-CM5-0', 'IPSL-CM6A-LR', 'MIROC6',
          'MPI-ESM1-2-HR', 'MRI-ESM2-0', 'NorESM2-MM', 'UKESM1-0-LL']

# calculate the spatial mean and standard deviation for all periods for each model
for model in models:

    # define the pathways to original CMIP6 near surface temperature data
    pw_585 = glob.glob(f'/home/ucfagtj/DATA/Dissertation/Data/Temperature/working_files/ssp585/*{model}*')
    pw_370 = glob.glob(f'/home/ucfagtj/DATA/Dissertation/Data/Temperature/working_files/ssp370/*{model}*')
    pw_245 = glob.glob(f'/home/ucfagtj/DATA/Dissertation/Data/Temperature/working_files/ssp245/*{model}*')
    pw_126 = glob.glob(f'/home/ucfagtj/DATA/Dissertation/Data/Temperature/working_files/ssp126/*{model}*')
    pw_hist = glob.glob(f'/home/ucfagtj/DATA/Dissertation/Data/Temperature/working_files/historic/*{model}*')
   
    # compute the spatial mean and standard deviation of the annual near surface temperature
    ds_585 = ann_tas(merge_arrays_via_time(pw_585), '2071', '2100')
    ds_370 = ann_tas(merge_arrays_via_time(pw_370), '2071', '2100')
    ds_245 = ann_tas(merge_arrays_via_time(pw_245), '2071', '2100')
    ds_126 = ann_tas(merge_arrays_via_time(pw_126), '2071', '2100')
    ds_preind = ann_tas(merge_arrays_via_time(pw_hist), '1851', '1900')
    ds_cur = ann_tas(merge_arrays_via_time(pw_hist), '1981', '2010')
    
    # define pathways to save Dataset objects
    if model == 'MPI-ESM1-2-HR': # slight workaround due to variation of MPI modelling group names
        spw_585 = f'/home/ucfagtj/DATA/Dissertation/Data/Temperature/Annual/{ds_585.experiment}' + \
                  f'/MPI-M/{ds_585.model}' + \
                  f'/{ds_585.experiment}_{ds_585.model}_ann_tas_{ds_585.sample_start_year}-{ds_585.sample_end_year}.nc'
        spw_370 = f'/home/ucfagtj/DATA/Dissertation/Data/Temperature/Annual/{ds_370.experiment}' + \
                  f'/MPI-M/{ds_370.model}' + \
                  f'/{ds_370.experiment}_{ds_370.model}_ann_tas_{ds_370.sample_start_year}-{ds_370.sample_end_year}.nc'
        spw_245 = f'/home/ucfagtj/DATA/Dissertation/Data/Temperature/Annual/{ds_245.experiment}' + \
                  f'/MPI-M/{ds_245.model}' + \
                  f'/{ds_245.experiment}_{ds_245.model}_ann_tas_{ds_245.sample_start_year}-{ds_245.sample_end_year}.nc'
        spw_126 = f'/home/ucfagtj/DATA/Dissertation/Data/Temperature/Annual/{ds_126.experiment}' + \
                  f'/MPI-M/{ds_126.model}' + \
                  f'/{ds_126.experiment}_{ds_126.model}_ann_tas_{ds_126.sample_start_year}-{ds_126.sample_end_year}.nc'
        spw_cur = '/home/ucfagtj/DATA/Dissertation/Data/Temperature/Annual/current' + \
                  f'/MPI-M/{ds_cur.model}' + \
                  f'/current_{ds_cur.model}_ann_tas_{ds_cur.sample_start_year}-{ds_cur.sample_end_year}.nc'
        spw_preind = '/home/ucfagtj/DATA/Dissertation/Data/Temperature/Annual/pre_ind' + \
                     f'/MPI-M/{ds_preind.model}' + \
                     f'/pre_ind_{ds_preind.model}_ann_tas_{ds_preind.sample_start_year}' + \
                     f'-{ds_preind.sample_end_year}.nc'
        
    else:
        spw_585 = f'/home/ucfagtj/DATA/Dissertation/Data/Temperature/Annual/{ds_585.experiment}' + \
                  f'/{ds_585.modelling_group_id}/{ds_585.model}' + \
                  f'/{ds_585.experiment}_{ds_585.model}_ann_tas_{ds_585.sample_start_year}-{ds_585.sample_end_year}.nc'
        spw_370 = f'/home/ucfagtj/DATA/Dissertation/Data/Temperature/Annual/{ds_370.experiment}' + \
                  f'/{ds_370.modelling_group_id}/{ds_370.model}' + \
                  f'/{ds_370.experiment}_{ds_370.model}_ann_tas_{ds_370.sample_start_year}-{ds_370.sample_end_year}.nc'
        spw_245 = f'/home/ucfagtj/DATA/Dissertation/Data/Temperature/Annual/{ds_245.experiment}' + \
                  f'/{ds_245.modelling_group_id}/{ds_245.model}' + \
                  f'/{ds_245.experiment}_{ds_245.model}_ann_tas_{ds_245.sample_start_year}-{ds_245.sample_end_year}.nc'
        spw_126 = f'/home/ucfagtj/DATA/Dissertation/Data/Temperature/Annual/{ds_126.experiment}' + \
                  f'/{ds_126.modelling_group_id}/{ds_126.model}' + \
                  f'/{ds_126.experiment}_{ds_126.model}_ann_tas_{ds_126.sample_start_year}-{ds_126.sample_end_year}.nc'
        spw_cur = '/home/ucfagtj/DATA/Dissertation/Data/Temperature/Annual/current' + \
                  f'/{ds_cur.modelling_group_id}/{ds_cur.model}' + \
                  f'/current_{ds_cur.model}_ann_tas_{ds_cur.sample_start_year}-{ds_cur.sample_end_year}.nc'
        spw_preind = '/home/ucfagtj/DATA/Dissertation/Data/Temperature/Annual/pre_ind' + \
                     f'/{ds_preind.modelling_group_id}/{ds_preind.model}' + \
                     f'/pre_ind_{ds_preind.model}_ann_tas_{ds_preind.sample_start_year}' + \
                     f'-{ds_preind.sample_end_year}.nc'
    
    # save Dataset objects to defined pathways
    ds_585.to_netcdf(spw_585, mode = 'w')
    ds_370.to_netcdf(spw_370, mode = 'w')
    ds_245.to_netcdf(spw_245, mode = 'w')
    ds_126.to_netcdf(spw_126, mode = 'w')
    ds_cur.to_netcdf(spw_cur, mode = 'w')
    ds_preind.to_netcdf(spw_preind, mode = 'w')

    # close Dataset objects
    ds_585.close(), ds_370.close(), ds_245.close(),
    ds_126.close(), ds_cur.close(), ds_preind.close()
    
    print(f'{model} done.')

ACCESS-CM2 done.
ACCESS-ESM1-5 done.
AWI-CM-1-1-MR done.
BCC-CSM2-MR done.
CanESM5 done.
CNRM-CM6-1 done.
GFDL-ESM4 done.
INM-CM4-8 done.
INM-CM5-0 done.
IPSL-CM6A-LR done.
MIROC6 done.
MPI-ESM1-2-HR done.
MRI-ESM2-0 done.
NorESM2-MM done.
UKESM1-0-LL done.


### Spatial Mean and the Standard Deviation of  Annual Near Surface Temperature Anomaly: All Models

Raw monthly near surface temperature data has been accessed from the CEDA archives via JASMIN resources, and will be used to determine spatial mean annual near surface temperature anomaly and its associated standard deviation for a range of CMIP6 models.

Metacode:

1. Define baseline period to obtain the pre-industrial climatology from.
2. Compute the mean monthly temperatures across baseline period.
3. Compute the temperature anomalies in the projections by negating the baseline monthly values.
4. Compute the mean annual temperature anomaly and the standard deviation.
5. Form a Dataset object with these two variables.
6. Save Dataset object. (Ensure anomaly and 1851-1900 are in file name.)

As essentially only working out a time average across cells, no need for weighting to account for change in cell area with latitude.

In [18]:
def ann_anom_tas(bl_arr, pj_arr, start_year, end_year):
    
    '''
    Summary:
    --------
    Computes the spatial mean and standard deviation of the annual surface temperature anomaly 
    from monthly data across a specified range. Anomaly is relative to a pre-industrial baseline
    (1851-1900)
    
    Paramters:
    ----------
    bl_arr: xarray Dataset object
            xarray object must be contain monthly interval with surface temperature variable (tas)
            xarray object must be chronologically ordered
            xarray object must contain at least data between 1851-1900
            
    pj_arr: xarray Dataset object
            xarray object must be contain monthly interval with surface temperature variable (tas)
            xarray object must be chronologically ordered
               
    start_year: str
                starting year of the temporal range to be used
    
    end_year: str
              end year of the temporal range to be used
    
    Returns:
    --------
    anom_tas: xarray Dataset object
                  
    '''
    # load air surface temperature variable from both projection and baseline arrays
    pj_tas_arr, bl_tas_arr = pj_arr.tas, bl_arr.tas

    # convert temperautre from Kevlin to degree Celcius
    pj_tas_arr, bl_tas_arr = pj_tas_arr - 273.15, bl_tas_arr - 273.15

    # deterime baseline monthly temperature values; baseline period is 1851-1900
    bl_tas_arr = bl_tas_arr.sel(time = slice('1851', '1900', 1)) # use slice to index the time dimension
    monthly_bl = bl_tas_arr.groupby('time.month').mean('time', skipna = True)

    # restrict projection array to specified years
    pj_tas_arr = pj_tas_arr.sel(time = slice(start_year, end_year, 1))

    # compute monthly anomalies across entire period
    anomalies = pj_tas_arr.groupby('time.month') - monthly_bl

    # compute the yearly spatial mean anomaly for each year
    anomalies = anomalies.groupby('time.year').mean()

    # compute the spatial mean annual anomaly
    ann_tas_anom = anomalies.mean(dim = 'year', skipna = True)

    # compute the standard deviation of the spatial mean annual anomaly; the intra-annual variability
    std_dev_ann_tas_anom = anomalies.std(dim = 'year', skipna = True)

    # define some attributes for the mean and standard deviation DataArray objects
    ann_tas_anom.attrs['units'], ann_tas_anom.attrs['sample_start_year'] = 'degrees Celcius', start_year
    ann_tas_anom.attrs['sample_end_year'], ann_tas_anom.attrs['model'] = end_year, pj_arr.source_id
    ann_tas_anom.attrs['modelling_group'] = pj_arr.institution
    ann_tas_anom.attrs['modelling_group_id'] = pj_arr.institution_id
    ann_tas_anom.attrs['experiment'] = pj_arr.experiment_id
    ann_tas_anom.attrs['realisation'] =  pj_arr.variant_label
    std_dev_ann_tas_anom.attrs['units'], std_dev_ann_tas_anom.attrs['sample_start_year'] = 'degrees Celcius', start_year
    std_dev_ann_tas_anom.attrs['sample_end_year'], std_dev_ann_tas_anom.attrs['model'] = end_year, pj_arr.source_id
    std_dev_ann_tas_anom.attrs['modelling_group'] = pj_arr.institution
    std_dev_ann_tas_anom.attrs['modelling_group_id'] = pj_arr.institution_id
    std_dev_ann_tas_anom.attrs['experiment'] = pj_arr.experiment_id
    std_dev_ann_tas_anom.attrs['realisation'] =  pj_arr.variant_label

    # create Dataset object to hold anomaly mean and standard deviation DataArray objects
    anom_tas = xr.Dataset({'ann_tas_anom': ann_tas_anom,
                           'std_dev_ann_tas_anom': std_dev_ann_tas_anom})

    # define some attributes for the Dataset object
    anom_tas.attrs['comment'] = 'spatial annual anomaly mean and standard deviation of near surface temperature.'
    anom_tas.attrs['units'] = 'degrees Celcius'
    anom_tas.attrs['sample_start_year'], anom_tas.attrs['sample_end_year'] = start_year, end_year
    anom_tas.attrs['model'], anom_tas.attrs['modelling_group'] = pj_arr.source_id, pj_arr.institution
    anom_tas.attrs['modelling_group_id'], anom_tas.attrs['experiment'] = pj_arr.institution_id, pj_arr.experiment_id
    anom_tas.attrs['realisation'], anom_tas.attrs['created'] = pj_arr.variant_label, time.ctime()

    # close DataArray objects
    pj_tas_arr.close(), bl_tas_arr.close(), pj_arr.close(), bl_arr.close()
    monthly_bl.close(), anomalies.close(), ann_tas_anom.close(), std_dev_ann_tas_anom.close()
    
    return anom_tas

In [20]:
# define a list containing the CMIP6 models used in study
models = ['ACCESS-CM2', 'ACCESS-ESM1-5', 'AWI-CM-1-1-MR', 'BCC-CSM2-MR', 'CanESM5',
          'CNRM-CM6-1', 'GFDL-ESM4', 'INM-CM4-8', 'INM-CM5-0', 'IPSL-CM6A-LR', 'MIROC6',
          'MPI-ESM1-2-HR', 'MRI-ESM2-0', 'NorESM2-MM', 'UKESM1-0-LL']

# calculate the spatial mean anomaly and standard deviation for all periods for each model
for model in models:

    # define the pathways to original CMIP6 near surface temperature data
    pw_585 = glob.glob(f'/home/ucfagtj/DATA/Dissertation/Data/Temperature/working_files/ssp585/*{model}*')
    pw_370 = glob.glob(f'/home/ucfagtj/DATA/Dissertation/Data/Temperature/working_files/ssp370/*{model}*')
    pw_245 = glob.glob(f'/home/ucfagtj/DATA/Dissertation/Data/Temperature/working_files/ssp245/*{model}*')
    pw_126 = glob.glob(f'/home/ucfagtj/DATA/Dissertation/Data/Temperature/working_files/ssp126/*{model}*')
    pw_hist = glob.glob(f'/home/ucfagtj/DATA/Dissertation/Data/Temperature/working_files/historic/*{model}*')
   
    # compute the spatial mean anomaly and standard deviation of the annual near surface temperature
    ds_585 = ann_anom_tas(merge_arrays_via_time(pw_hist), merge_arrays_via_time(pw_585), '2071', '2100')
    ds_370 = ann_anom_tas(merge_arrays_via_time(pw_hist), merge_arrays_via_time(pw_370), '2071', '2100')
    ds_245 = ann_anom_tas(merge_arrays_via_time(pw_hist), merge_arrays_via_time(pw_245), '2071', '2100')
    ds_126 = ann_anom_tas(merge_arrays_via_time(pw_hist), merge_arrays_via_time(pw_126), '2071', '2100')
    ds_cur = ann_anom_tas(merge_arrays_via_time(pw_hist), merge_arrays_via_time(pw_hist), '1981', '2010')
    
    # define pathways to save Dataset objects
    if model == 'MPI-ESM1-2-HR': # slight workaround due to variation of MPI modelling group names
        spw_585 = f'/home/ucfagtj/DATA/Dissertation/Data/Temperature/Annual/{ds_585.experiment}' + \
                  f'/MPI-M/{ds_585.model}' + \
                  f'/{ds_585.experiment}_{ds_585.model}_anomaly_tas_{ds_585.sample_start_year}-{ds_585.sample_end_year}' + \
                  '_relative_1851-1900.nc'
        spw_370 = f'/home/ucfagtj/DATA/Dissertation/Data/Temperature/Annual/{ds_370.experiment}' + \
                  f'/MPI-M/{ds_370.model}' + \
                  f'/{ds_370.experiment}_{ds_370.model}_anomaly_tas_{ds_370.sample_start_year}-{ds_370.sample_end_year}' + \
                  '_relative_1851-1900.nc'
        spw_245 = f'/home/ucfagtj/DATA/Dissertation/Data/Temperature/Annual/{ds_245.experiment}' + \
                  f'/MPI-M/{ds_245.model}' + \
                  f'/{ds_245.experiment}_{ds_245.model}_anomaly_tas_{ds_245.sample_start_year}-{ds_245.sample_end_year}' + \
                  '_relative_1851-1900.nc'
        spw_126 = f'/home/ucfagtj/DATA/Dissertation/Data/Temperature/Annual/{ds_126.experiment}' + \
                  f'/MPI-M/{ds_126.model}' + \
                  f'/{ds_126.experiment}_{ds_126.model}_anomaly_tas_{ds_126.sample_start_year}-{ds_126.sample_end_year}' + \
                  '_relative_1851-1900.nc'
        spw_cur = '/home/ucfagtj/DATA/Dissertation/Data/Temperature/Annual/current' + \
                  f'/MPI-M/{ds_cur.model}' + \
                  f'/current_{ds_cur.model}_anomaly_tas_{ds_cur.sample_start_year}-{ds_cur.sample_end_year}' + \
                  '_relative_1851-1900.nc'
        
    else:
        spw_585 = f'/home/ucfagtj/DATA/Dissertation/Data/Temperature/Annual/{ds_585.experiment}' + \
                  f'/{ds_585.modelling_group_id}/{ds_585.model}' + \
                  f'/{ds_585.experiment}_{ds_585.model}_anomaly_tas_{ds_585.sample_start_year}-{ds_585.sample_end_year}' + \
                  '_relative_1851-1900.nc'
        spw_370 = f'/home/ucfagtj/DATA/Dissertation/Data/Temperature/Annual/{ds_370.experiment}' + \
                  f'/{ds_370.modelling_group_id}/{ds_370.model}' + \
                  f'/{ds_370.experiment}_{ds_370.model}_anomaly_tas_{ds_370.sample_start_year}-{ds_370.sample_end_year}' + \
                  '_relative_1851-1900.nc'
        spw_245 = f'/home/ucfagtj/DATA/Dissertation/Data/Temperature/Annual/{ds_245.experiment}' + \
                  f'/{ds_245.modelling_group_id}/{ds_245.model}' + \
                  f'/{ds_245.experiment}_{ds_245.model}_anomaly_tas_{ds_245.sample_start_year}-{ds_245.sample_end_year}' + \
                  '_relative_1851-1900.nc'
        spw_126 = f'/home/ucfagtj/DATA/Dissertation/Data/Temperature/Annual/{ds_126.experiment}' + \
                  f'/{ds_126.modelling_group_id}/{ds_126.model}' + \
                  f'/{ds_126.experiment}_{ds_126.model}_anomaly_tas_{ds_126.sample_start_year}-{ds_126.sample_end_year}' + \
                  '_relative_1851-1900.nc'
        spw_cur = '/home/ucfagtj/DATA/Dissertation/Data/Temperature/Annual/current' + \
                  f'/{ds_cur.modelling_group_id}/{ds_cur.model}' + \
                  f'/current_{ds_cur.model}_anomaly_tas_{ds_cur.sample_start_year}-{ds_cur.sample_end_year}' + \
                  '_relative_1851-1900.nc'
    
    # save Dataset objects to defined pathways
    ds_585.to_netcdf(spw_585, mode = 'w')
    ds_370.to_netcdf(spw_370, mode = 'w')
    ds_245.to_netcdf(spw_245, mode = 'w')
    ds_126.to_netcdf(spw_126, mode = 'w')
    ds_cur.to_netcdf(spw_cur, mode = 'w')
    ds_preind.to_netcdf(spw_preind, mode = 'w')

    # close Dataset objects
    ds_585.close(), ds_370.close(), ds_245.close(),
    ds_126.close(), ds_cur.close(), ds_preind.close()
    
    print(f'{model} done.')

ACCESS-CM2 done.
ACCESS-ESM1-5 done.
AWI-CM-1-1-MR done.
BCC-CSM2-MR done.
CanESM5 done.
CNRM-CM6-1 done.
GFDL-ESM4 done.
INM-CM4-8 done.
INM-CM5-0 done.
IPSL-CM6A-LR done.
MIROC6 done.
MPI-ESM1-2-HR done.
MRI-ESM2-0 done.
NorESM2-MM done.
UKESM1-0-LL done.


In [8]:
pw_585 = glob.glob(f'/home/ucfagtj/DATA/Dissertation/Data/Temperature/working_files/ssp585/*UKESM*-2049*')
x = xr.open_dataset(pw_585[0])
print(x.tas)
x.close()

<xarray.DataArray 'tas' (time: 420, lat: 144, lon: 192)>
[11612160 values with dtype=float32]
Coordinates:
  * time     (time) object 2015-01-16 00:00:00 ... 2049-12-16 00:00:00
  * lat      (lat) float64 -89.38 -88.12 -86.88 -85.62 ... 86.88 88.12 89.38
  * lon      (lon) float64 0.9375 2.812 4.688 6.562 ... 353.4 355.3 357.2 359.1
    height   float64 ...
Attributes:
    standard_name:  air_temperature
    long_name:      Near-Surface Air Temperature
    comment:        near-surface (usually, 2 meter) air temperature
    units:          K
    original_name:  mo: (stash: m01s03i236, lbproc: 128)
    cell_methods:   area: time: mean
    cell_measures:  area: areacella
    history:        2019-04-18T14:32:46Z altered by CMOR: Treated scalar dime...


### Regridding of Model Output: 1.0x1.0 and 5.0x5.0 degree latitude and longitude grids

Metacode:
1. Define desired resolution of data to be regridded to.
2. Read in original resolution data.
3. Compute a "regridder" which will transform the data from one resolution to another.
4. Save regridded data to the directory containing the original resolution data.

Data regridded to both 1x1 and 5.0x5.0 grids.

In [ ]:
# define absolute and anomaly near surface temperature data to regrid; both end in '0.nc'
f_pws = glob.glob('/home/ucfagtj/DATA/Dissertation/Data/Temperature/Annual/*/*/*/*0.nc')

# define desired output grid; 1.0 x 1.0 degrees
grid_out = xr.Dataset({'lat': (['lat'], np.arange(-89.5, 90.5, 1.)),
                       'lon': (['lon'], np.arange(-179.5, 180.5, 1.))})

for pw in f_pws:
    
    # load in near surface temperature data in the original resolution
    grid_in = xr.open_dataset(pw)
    
    # compute the "regridder" file which will define the weighting to apply
    regridder = xe.Regridder(grid_in, grid_out,
                             method = 'bilinear', # interpolation method
                             periodic = True) # required for global girds; prevents blank data on meridian
    
    # apply the weighting matrix to transform the data to new resolution
    rg_data = regridder(grid_in)
    
    # clear regridder file from being saved
    regridder.clean_weight_file()
    
    # define some atrributes for the new resolution Dataset object
    rg_data.attrs['units'] = grid_in.units
    rg_data.attrs['period'] = f'{grid_in.sample_start_year}-{grid_in.sample_end_year}'
    rg_data.attrs['model'], rg_data.attrs['modelling_group'] = grid_in.model, grid_in.modelling_group
    rg_data.attrs['experiment'], rg_data.attrs['realisation'] = grid_in.experiment, grid_in.realisation
    rg_data.attrs['resolution'], rg_data.attrs['created_on'] = 'lon-lat: 1x1 degrees', time.ctime()
    
    # save resulting dataset
    save_pw = pw[: -3] + '_1x1_res.nc'
    rg_data.to_netcdf(save_pw, mode = 'w')
    print(f'File saved: {save_pw.rsplit("/")[-1]}')

    # close open Datasets
    grid_out.close(), grid_in.close(), rg_data.close()

In [12]:
# define absolute near surface temperature data to regrid; both end in '0.nc'
f_pws = glob.glob('/home/ucfagtj/DATA/Dissertation/Data/Temperature/Annual/*/*/*/*_ann_tas_*0.nc')

# define desired output grid; 5.0 x 5.0 latitude-longitude grid
grid_out = xr.Dataset({'lat': (['lat'], np.arange(-87.5, 92.5, 5.)),
                       'lon': (['lon'], np.arange(-177.5, 182.5, 5.))})

for pw in f_pws:
    
    # load in near surface temperature data in the original resolution
    grid_in = xr.open_dataset(pw)
    
    # compute the "regridder" file which will define the weighting to apply
    regridder = xe.Regridder(grid_in, grid_out,
                             method = 'bilinear', # interpolation method
                             periodic = True) # required for global girds; prevents blank data on meridian
    
    # apply the weighting matrix to transform the data to new resolution
    rg_data = regridder(grid_in)
    
    # clear regridder file from being saved
    regridder.clean_weight_file()
    
    # define some atrributes for the new resolution Dataset object
    rg_data.attrs['units'] = grid_in.units
    rg_data.attrs['period'] = f'{grid_in.sample_start_year}-{grid_in.sample_end_year}'
    rg_data.attrs['model'], rg_data.attrs['modelling_group'] = grid_in.model, grid_in.modelling_group
    rg_data.attrs['experiment'], rg_data.attrs['realisation'] = grid_in.experiment, grid_in.realisation
    rg_data.attrs['resolution'], rg_data.attrs['created_on'] = 'lat-lon: 5x5 degrees', time.ctime()
    
    # save resulting dataset
    save_pw = pw[: -3] + '_5x5_res.nc'
    rg_data.to_netcdf(save_pw, mode = 'w')
    print(f'File saved: {save_pw.rsplit("/")[-1]}')

    # close open Datasets
    grid_out.close(), grid_in.close(), rg_data.close()

Create weight file: bilinear_160x320_36x72_peri.nc
using dimensions ('lat', 'lon') from data variable mean_tas as the horizontal dimensions for this dataset.
Remove file bilinear_160x320_36x72_peri.nc
File saved: ssp126_BCC-CSM2-MR_ann_tas_2071-2100_5x5_res.nc
Create weight file: bilinear_144x192_36x72_peri.nc
using dimensions ('lat', 'lon') from data variable mean_tas as the horizontal dimensions for this dataset.
Remove file bilinear_144x192_36x72_peri.nc
File saved: ssp126_UKESM1-0-LL_ann_tas_2071-2100_5x5_res.nc
Create weight file: bilinear_160x320_36x72_peri.nc
using dimensions ('lat', 'lon') from data variable mean_tas as the horizontal dimensions for this dataset.
Remove file bilinear_160x320_36x72_peri.nc
File saved: ssp126_MRI-ESM2-0_ann_tas_2071-2100_5x5_res.nc
Create weight file: bilinear_64x128_36x72_peri.nc
using dimensions ('lat', 'lon') from data variable mean_tas as the horizontal dimensions for this dataset.
Remove file bilinear_64x128_36x72_peri.nc
File saved: ssp126_

using dimensions ('lat', 'lon') from data variable mean_tas as the horizontal dimensions for this dataset.
Remove file bilinear_64x128_36x72_peri.nc
File saved: ssp370_CanESM5_ann_tas_2071-2100_5x5_res.nc
Create weight file: bilinear_143x144_36x72_peri.nc
using dimensions ('lat', 'lon') from data variable mean_tas as the horizontal dimensions for this dataset.
Remove file bilinear_143x144_36x72_peri.nc
File saved: ssp370_IPSL-CM6A-LR_ann_tas_2071-2100_5x5_res.nc
Create weight file: bilinear_128x256_36x72_peri.nc
using dimensions ('lat', 'lon') from data variable mean_tas as the horizontal dimensions for this dataset.
Remove file bilinear_128x256_36x72_peri.nc
File saved: ssp370_CNRM-CM6-1_ann_tas_2071-2100_5x5_res.nc
Create weight file: bilinear_160x320_36x72_peri.nc
using dimensions ('lat', 'lon') from data variable mean_tas as the horizontal dimensions for this dataset.
Remove file bilinear_160x320_36x72_peri.nc
File saved: ssp370_BCC-CSM2-MR_ann_tas_2071-2100_5x5_res.nc
Create weigh

using dimensions ('lat', 'lon') from data variable mean_tas as the horizontal dimensions for this dataset.
Remove file bilinear_120x180_36x72_peri.nc
File saved: pre_ind_INM-CM5-0_ann_tas_1851-1900_5x5_res.nc
Create weight file: bilinear_120x180_36x72_peri.nc
using dimensions ('lat', 'lon') from data variable mean_tas as the horizontal dimensions for this dataset.
Remove file bilinear_120x180_36x72_peri.nc
File saved: pre_ind_INM-CM4-8_ann_tas_1851-1900_5x5_res.nc
Create weight file: bilinear_128x256_36x72_peri.nc
using dimensions ('lat', 'lon') from data variable mean_tas as the horizontal dimensions for this dataset.
Remove file bilinear_128x256_36x72_peri.nc
File saved: pre_ind_MIROC6_ann_tas_1851-1900_5x5_res.nc
Create weight file: bilinear_160x320_36x72_peri.nc
using dimensions ('lat', 'lon') from data variable mean_tas as the horizontal dimensions for this dataset.
Remove file bilinear_160x320_36x72_peri.nc
File saved: pre_ind_MRI-ESM2-0_ann_tas_1851-1900_5x5_res.nc
Create weight

### Ensemble Spatial Mean and Standard Deviation of Annual Near Surface Temperature

Metacode:
1. Define pathway for spatial annual near surface temperature data.
2. For each period/scenario, concatenate all model output along a new dimension. (Model output is in the form of two DataArray objects; spatial mean annual near surface temperature and standard deviation of annual near surface temperature/inter-annual variability.)
3. Compute the spatial mean along this new dimension for both DataArray objects (ensemble spatial means of annual near surface temperature and inter-annual near surface temperature variability).
4. Compute the spatial standard deviation along this new dimension for spatial mean DataArray object (measure of ensemble variance).
5. Form Dataset object from ensemble means and standard deviation DataArray objects from all periods/scenarios.
6. Save Dataset object to suitable directory.

Above is to be applied to both absolute and anomaly tropical nights 1x1 degree resolution data.

In [32]:
anom_cur = glob.glob('/home/ucfagtj/DATA/Dissertation/Data/Temperature/Annual/current/*/*/*anomaly*_1x1*')
anom_126 = glob.glob('/home/ucfagtj/DATA/Dissertation/Data/Temperature/Annual/ssp126/*/*/*anomaly*_1x1*')
anom_245 = glob.glob('/home/ucfagtj/DATA/Dissertation/Data/Temperature/Annual/ssp245/*/*/*anomaly*_1x1*')
anom_370 = glob.glob('/home/ucfagtj/DATA/Dissertation/Data/Temperature/Annual/ssp370/*/*/*anomaly*_1x1*')
anom_585 = glob.glob('/home/ucfagtj/DATA/Dissertation/Data/Temperature/Annual/ssp585/*/*/*anomaly*_1x1*')
abso_pre = glob.glob('/home/ucfagtj/DATA/Dissertation/Data/Temperature/Annual/pre_ind/*/*/*ann_tas*_1x1*')
abso_cur = glob.glob('/home/ucfagtj/DATA/Dissertation/Data/Temperature/Annual/current/*/*/*ann_tas*_1x1*')
abso_126 = glob.glob('/home/ucfagtj/DATA/Dissertation/Data/Temperature/Annual/ssp126/*/*/*ann_tas*_1x1*')
abso_245 = glob.glob('/home/ucfagtj/DATA/Dissertation/Data/Temperature/Annual/ssp245/*/*/*ann_tas*_1x1*')
abso_370 = glob.glob('/home/ucfagtj/DATA/Dissertation/Data/Temperature/Annual/ssp370/*/*/*ann_tas*_1x1*')
abso_585 = glob.glob('/home/ucfagtj/DATA/Dissertation/Data/Temperature/Annual/ssp585/*/*/*ann_tas*_1x1*')

f_pws_list = [anom_cur, anom_126, anom_245, anom_370, anom_585,
              abso_pre, abso_cur, abso_126, abso_245, abso_370, abso_585]

In [60]:
# create Dataset object to store the spatial ensemble variables of surface temperature
ens_ds = xr.Dataset()

for pws_group in f_pws_list:
    
    # load in a single model output; remaining model outputs will be concatenated to this
    x0 = xr.open_dataset(pws_group[0], use_cftime = True)
    models = [x0.model] # note model used
    
    # ensure all Datasets use same coordinate set by losing excess height coordinate if present
    if 'height' in x0.coords:
        x0 = x0.drop('height')
    
    # concatenate remaining model ouputs along a new dimension
    for pw in pws_group[1: ]:
        x = xr.open_dataset(pw, use_cftime = True)
        if 'height' in x.coords: # drop excess height coordinate
            x = x.drop('height')
        x0 = xr.concat([x0, x], dim = 'model')
        
        # note model concatenated
        models += [x.model]
        
        # close Dataset object
        x.close()
        del(x)
    
    # update attribute information to concatenated Dataset
    x0.attrs['model_index'] = models # add names of model concatenated in order of concatenation;
                                     # index of model dimension will match names in list
    if x0.experiment == 'historical' and x0.period == '1981-2010': # distinguish between historical periods
        x0.attrs['experiment'] = 'current'
    if x0.experiment == 'historical' and x0.period == '1851-1900':
        x0.attrs['experiment'] = 'pre_ind'
    if 'anomaly' in pw:
        x0.attrs['data_type'] = 'anomaly'
    if 'ann_tas' in pw:
        x0.attrs['data_type'] = 'absolute'
    del(x0.attrs['model'], x0.attrs['modelling_group'], x0.attrs['realisation'])
    
    # save Dataset object containing the concatenated model output for specfic group
    save_pw = '/home/ucfagtj/DATA/Dissertation/Data/Temperature/processed/' + \
              f'{x0.experiment}_{x0.data_type[0: 4]}_all_models.nc'
    x0.to_netcdf(save_pw, 'w')
    print(f'File saved: {save_pw.split("/")[-1]}')
    del(save_pw)
    
    # create a dictionary object storing attribute information for new objects
    attrs_info = {'models': models, 'models_used': len(models), 'regird_method': x0.regrid_method,
                  'data_type': x0.data_type, 'experiment': x0.experiment, 'units': x0.units,
                  'period': x0.period, 'resolution': x0.resolution}

    # compute the ensemble spatial annual near surface temperature variables and add attribute information
    if 'anomaly' in x0.data_type:
        ens_mean = x0.ann_tas_anom.mean(dim = 'model', skipna = True).assign_attrs(attrs_info)
        ens_intra = x0.std_dev_ann_tas_anom.mean(dim = 'model', skipna = True).assign_attrs(attrs_info)
        ens_std = x0.ann_tas_anom.std(dim = 'model', skipna = True).assign_attrs(attrs_info)
        ens_qua = x0.ann_tas_anom.quantile([0.05, 0.25, 0.75, 0.95], dim = 'model',
                                               interpolation = 'linear').assign_attrs(attrs_info)
        ens_medi = x0.ann_tas_anom.median(dim = 'model', skipna = True).assign_attrs(attrs_info)
    
    elif 'absolute' in x0.data_type:
        ens_mean = x0.mean_tas.mean(dim = 'model', skipna = True).assign_attrs(attrs_info)
        ens_intra = x0.std_dev_tas.mean(dim = 'model', skipna = True).assign_attrs(attrs_info)
        ens_std = x0.mean_tas.std(dim = 'model', skipna = True).assign_attrs(attrs_info)
        ens_qua = x0.mean_tas.quantile([0.05, 0.25, 0.75, 0.95], dim = 'model',
                                          interpolation = 'linear').assign_attrs(attrs_info)
        ens_medi = x0.mean_tas.median(dim = 'model', skipna = True).assign_attrs(attrs_info)
    
    else:
        raise ValueError('check data type attribute of Datasets being averaged.')
        
    # add DataArray objects to ensemble Dataset object
    ens_ds = ens_ds.assign({f'{x0.experiment}_mean_ann_tas_{x0.data_type[0: 4]}': ens_mean})
    ens_ds = ens_ds.assign({f'{x0.experiment}_intrvar_ann_tas_{x0.data_type[0: 4]}': ens_intra})
    ens_ds = ens_ds.assign({f'{x0.experiment}_std_ann_tas_{x0.data_type[0: 4]}': ens_std})
    ens_ds = ens_ds.assign({f'{x0.experiment}_quantiles_ann_tas_{x0.data_type[0: 4]}': ens_qua})
    ens_ds = ens_ds.assign({f'{x0.experiment}_medi_ann_tas_{x0.data_type[0: 4]}': ens_medi})
    
# update attribute informaton to Dataset object
ens_ds = ens_ds.assign_attrs(attrs_info)
del(ens_ds.attrs['data_type'], ens_ds.attrs['period'], ens_ds.attrs['experiment'])
ens_ds.attrs['description'] = '"mean_ann_tas" = spatial ensemble mean annual near surface temperature; ' +\
                              '"intrvar_ann_tas" = spatial ensemble mean model inter-annual variabiltiy in '+\
                              'annual near surface temperature; "std_ann_tas" = spatial ensemble standard '+\
                              'deviation in annual near surface temperature; "quantiles_ann_tas" = 5th, 25th, 75th '+\
                              'and 95th percentiles of ensemble annual near surface temperature; "medi_ann_tas" = '+\
                              'spatial ensemble median annual near surface temperatures.'
    
# save Dataset object
save_pw = '/home/ucfagtj/DATA/Dissertation/Data/Temperature/processed/ensemble_tas.nc'
ens_ds.to_netcdf(save_pw, 'w')
print(f'File saved: {save_pw.split("/")[-1]}')

# close DataArray/set objects
x0.close(), ens_ds.close(), ens_mean.close(), ens_intra.close(), ens_std.close(),
ens_qua.close(), ens_medi.close()

File saved: current_anom_all_models.nc
File saved: ssp126_anom_all_models.nc
File saved: ssp245_anom_all_models.nc
File saved: ssp370_anom_all_models.nc
File saved: ssp585_anom_all_models.nc
File saved: pre_ind_abso_all_models.nc
File saved: current_abso_all_models.nc
File saved: ssp126_abso_all_models.nc
File saved: ssp245_abso_all_models.nc
File saved: ssp370_abso_all_models.nc
File saved: ssp585_abso_all_models.nc
File saved: ensemble_tas.nc


(None, None)

A multi-model ensemble for the current period only is to be regridded to 5x5 latitude-longitude grid for use with an observational dataset (CRUTEM4) for model evaluation.

In [21]:
# create Dataset object to store the spatial ensemble mean annual tropical nights variable
ens_ds = xr.Dataset()

# define pathway to current absolute 5x5 resolution model output
abso_cur = glob.glob('/home/ucfagtj/DATA/Dissertation/Data/Temperature/Annual/current/*/*/*ann_tas*5x5*')
    
# load in a single model output; remaining model outputs will be concatenated to this
x0 = xr.open_dataset(abso_cur[0], use_cftime = True)
models = [x0.model] # note model used

# ensure all Datasets use same coordinate set by losing excess height coordinate if present
if 'height' in x0.coords:
    x0 = x0.drop('height')
    
# concatenate remaining model ouputs along a new dimension
for pw in abso_cur[1: ]:
    x = xr.open_dataset(pw, use_cftime = True)
    x0 = xr.concat([x0, x], dim = 'model')
    if 'height' in x.coords: # drop excess height coordinate
        x = x.drop('height')
        
    # note model concatenated
    models += [x.model]
        
    # close Dataset object
    x.close()
    del(x)
        
# update attribute information to concatenated Dataset
x0.attrs['model_index'] = models # add names of model concatenated in order of concatenation;
                                 # index of model dimension will match names in list
del(x0.attrs['model'], x0.attrs['modelling_group'], x0.attrs['realisation'])
        
# save Dataset object containing the concatenated model output for specfic group
save_pw = '/home/ucfagtj/DATA/Dissertation/Data/Temperature/processed/' + \
           'curret_abso_all_models_5x5_res.nc'
x0.to_netcdf(save_pw, 'w')
print(f'File saved: {save_pw.split("/")[-1]}')
del(save_pw)
    
# create a dictionary object storing attribute information for new objects
attrs_info = {'models': models, 'models_used': len(models), 'regird_method': x0.regrid_method,
              'data_type': 'absolute', 'experiment': x0.experiment,
              'year_sampled': '30', 'period': 'current', 'resolution': x0.resolution}

# compute the ensemble spatial annual tropical night variables and add attribute information
ens_mean = x0.mean_tas.mean(dim = 'model', skipna = True).assign_attrs(attrs_info)
ens_intra = x0.std_dev_tas.mean(dim = 'model', skipna = True).assign_attrs(attrs_info)
ens_std = x0.mean_tas.std(dim = 'model', skipna = True).assign_attrs(attrs_info)
ens_qua = x0.mean_tas.quantile([0.05, 0.25, 0.75, 0.95], dim = 'model',
                                interpolation = 'linear').assign_attrs(attrs_info)
ens_medi = x0.mean_tas.median(dim = 'model', skipna = True).assign_attrs(attrs_info)
       
# add DataArray objects to ensemble Dataset object
ens_ds = ens_ds.assign({'current_mean_ann_tas_abso': ens_mean})
ens_ds = ens_ds.assign({'current_intrvar_ann_tas_abso': ens_intra})
ens_ds = ens_ds.assign({'current_std_ann_tas_abso': ens_std})
ens_ds = ens_ds.assign({'current_quantiles_ann_tas_abso': ens_qua})
ens_ds = ens_ds.assign({'current_medi_ann_tas_abso': ens_medi})
    
# update attribute informaton to Dataset object
ens_ds = ens_ds.assign_attrs(attrs_info)
del(ens_ds.attrs['data_type'], ens_ds.attrs['period'], ens_ds.attrs['experiment'],
    ens_ds.attrs['year_sampled'])
ens_ds.attrs['description'] = '"mean_ann_tas" = spatial ensemble mean annual near surface temperature; ' +\
                              '"intrvar_ann_tas" = spatial ensemble mean model inter-annual variabiltiy in '+\
                              'annual near surface temperature; "std_ann_tas" = spatial ensemble standard '+\
                              'deviation in annual near surface temperature; "quantiles_ann_tas" = 5th, 25th, 75th '+\
                              'and 95th percentiles of ensemble annual near surface temperature; "medi_ann_tas" = '+\
                              'spatial ensemble median annual near surface temperatures.'

# save Dataset object
save_pw = '/home/ucfagtj/DATA/Dissertation/Data/Temperature/processed/ensemble_tas_5x5_res.nc'
ens_ds.to_netcdf(save_pw, 'w')
print(f'File saved: {save_pw.split("/")[-1]}')

# close DataArray/set objects
x0.close(), ens_ds.close(), ens_mean.close(), ens_intra.close(), ens_std.close(),
ens_qua.close(), ens_medi.close()

File saved: curret_abso_all_models_5x5_res.nc
File saved: ensemble_tas_5x5_res.nc


(None, None)

### Weighted Regional Ensemble Means

As regional and surface means will use raster cells of differing areas, a weighting must be applied such that cell values with a larger area contribute more towards the mean. For rectangular grid data, the cosine of the latitude is a good approximation to provide a weighting.

Metacode:

1. Create a weighted mean function. This function will take 3 inputs; a data array, a weights array, and dimension/s to compute the weighted mean over.
2. Define regions to calculate weighted means for.
3. Create a weights array using latitude cosine approximation. Use this weights array to assign zero weights to cells outside desired region.
4. Compute weighted means for each region for each DataArray object.
5. Populate a DataFrame object with weighted means.
6. Save DataFrame object using serialisation for efficiency ("pickling").

In [63]:
def weighted_mean(arr, weights, dim):
    '''
    Summary:
    --------
    Computes a weighted mean of an array along a dimension/dimensions of a DataArray object.
    
    Parameters:
    -----------
    arr: xarray DataArray object
         array containing values used to compute a weighted mean
         
    weights: xarray DataArray object
             array containing weights to be applied to the values within arr object
             
    dim: str or sequence of str
         dimension/s over which to compute the weighted mean
         
    Returns:
    --------
    weighted_mean: xarray DataArray oject
                   array containing weighted mean with the dimension/s mean calculated over removed.
                   
    '''
    
    # sum up the weighted sum of the values within the region specified by dim
    # recall matrix multiplication; cols of first equal rows of second; (10x2) * (2x6) = (10x6)
    # so weights array does not need to be same shape as data array
    weighted_sum = (arr * weights).sum(dim = dim, skipna = True)
    
    # sum up the values of the weights within the region specified by dim
    # define an array where weights of cells with valid and invalid values are preserved and NA respectively
    masked_weights = weights.where(arr.notnull()) 
    
    # sum up the weights of the valid cells
    sum_of_weights = masked_weights.sum(dim = dim, skipna = True)
    
    # as cannot divide by zero, set weights equal to zero as NA
    # the values of these cells will be removed from the mean in the weighted sum part (multiplied by 0)
    valid_weights = sum_of_weights != 0
    sum_of_weights = sum_of_weights.where(valid_weights)
    
    # compute weighted mean along the specified dimension/s
    weighted_mean = weighted_sum / sum_of_weights
    
    return weighted_mean

In [64]:
def region_average(arr, regions, land_only = True):
    '''
    Summary:
    --------
    Computes weighted mean of an array for various regions, as well for the globe, the ocean, 
    and the land with and without Antartica.
    
    Parameters:
    -----------
    arr: xarray DataArray object
         array encompassing the regions where the weighted means are computed
    
    regions: regionmask.Regions object
             regions to compute the weighted means over
        
    land_only : bool
                whether to mask out ocean points before calculating regional means
                default is True
        
    Returns:
    --------
    reg_ave : xarray DataArray object
              New DataArray with weighted mean over the whole globe, the ocean, the land, the 
              land without Antarctica, and all regions.
              Dimensions (n_regions + 4) x (additional dimensions no averaged over)
    
    '''
    # check that regions specified are an instance of regionmask.Regions;
    # essnetially checking all are an 'regionmask.Regions' object type
    if not isinstance(regions, regionmask.Regions):
        raise ValueError('specified regions must be a regionmask.Regions instance')
        
    # define names of regions to be used to index the various weighted means to be computed
    abbrevs = ['global', 'ocean', 'land', 'land_wo_antarctica']
    abbrevs = abbrevs + regions.abbrevs
    
    # define the IPCC numbers for each regions to index the various weighted means to be computed
    numbers = np.array(regions.numbers)
    
    # compute the latitude weights using the cosine approximation
    weight = np.cos(np.deg2rad(arr.lat))
    
    # define a land mask where land and sea cells are True and False respectively
    landmask = regionmask.defined_regions.natural_earth.land_110.mask(arr)
    landmask = landmask == 0
    
    # for land only, combine latitude weighting with landmask
    # result being that only cells over land have non-zero weights following latitude cosine approximation
    if land_only:
        wgt = weight * landmask
    
    # otherwise, combine latitude weighting with same shape as input array
    # result being that all cells, both land or ocean, will have a weight corresponding to its latitude
    # this weight will be used for regions containing both ocean and land cells
    else:
        wgt = xr.full_like(landmask, 1) * weight
    
    # define a region mask; cells given number for a given region (Europe = 1, Aus = 2 etc.)
    # cells that do not fall into a region, denoted as NaN
    mask = regions.mask(arr)
    
    # define a list to accumulate averages/weighted means
    ave = list()
    
    # compute global mean
    # weighting is simple cosine latitude weighting; no mask as averging entire surface/all cells
    a = weighted_mean(arr, dim = ('lat', 'lon'), weights = weight)
    ave.append(a)
    
    # compute global ocean mean
    # weighting is a cosine latitude weighting of only ocean cells; land cells all weighted as 0
    weights = (weight * (1.0 - landmask))
    a = weighted_mean(arr, dim = ('lat', 'lon'), weights = weights)
    ave.append(a)
    
    # compute global land mean
    # weighting is a cosine latitude weighting of only land cells; ocean cells weighted as 0
    weights = (weight * landmask)
    a = weighted_mean(arr, dim = ('lat', 'lon'), weights = weights)
    ave.append(a)
    
    # compute global land mean without Antarctica
    # weighting is a cosine latitude weighting of only land cells; ocean cells weighted as 0
    arr_selected = arr.sel(lat = slice(-60, None)) # remove Antarctica by removing low latitudes
    weights = (weight * landmask)
    a = weighted_mean(arr_selected, dim = ('lat', 'lon'), weights = weights)
    ave.append(a)
    
    #### Regional Weighted Means ####
    # Computing the specified regional averages is quicker using Groupby objects
    # Groupby objects use multi indexing of coordinates to state the raster cells of a given group
    # Multi indexing essentially concatenates the coordinates
    # (i.e. a cell with lat = -5 and lon = 25 denoted as lat_lon = -5, 25)
    # Using these "stacked" coordinates reduces the dimensions/shape of an array
    # (i.e. stacking lat and lon will change the 2D representation of these to 1D)
    
    # compute the region weighted means
    g = arr.groupby(mask) # group array into the different regions
    
    # create a new dimension of 'stacked coordinates'; moves from raster/grid format to 1D object
    # (unstacked dimensions -> lat = 10, lon = 12, value = 50 stacked dimension -> 10_12, value = 50)
    wgt_stacked = wgt.stack(stacked_lat_lon = ('lat', 'lon'))
    
    # apply stacked lat_lon weights to stacked Groupby object
    a = g.apply(weighted_mean, dim = ('stacked_lat_lon'), weights = wgt_stacked)

    ave.append(a.drop('region')) # drop the region information as want to use as dimension to merge averages
    
    # merge the list of weighted means DataArray objects into a single DataArray object
    arr = xr.concat(ave, dim = 'region')
    
    # shift region coordinates such that the numbers correspond to the regions
    # accounting for the 4 non-regional weighted means also computed
    numbers = np.arange(numbers.min() - 4, numbers.max() + 1)
    
    # add the abbreviations of the regions and update the numbers
    arr = arr.assign_coords(**{'abbrev': ('region', abbrevs), 'number': ('region', numbers)})
    
    # create a multi index
    arr = arr.set_index(region = ('abbrev', 'number'))
    
    return arr    

In [68]:
# load in spatial ensemble annual near surface temperature variables; has both absolute and anomaly
f_pw = '/home/ucfagtj/DATA/Dissertation/Data/Temperature/processed/ensemble_tas.nc'
data = xr.open_dataset(f_pw)

# define AR6 land regions to calculate weighted means for; must be regionmask objects
regions = regionmask.defined_regions.ar6.land

# create DataFrame object to store weighted mean
wm_df = pd.DataFrame()

# define column headers
col_names = ['global', 'ocean', 'land', 'land_wo_antarctica']
col_names = col_names + regions.abbrevs

# add column headers to DataDrame object
wm_df = wm_df.reindex(columns = col_names)

# create a list of the various DataArray objects
data_vars = data.data_vars.values() # .data_vars gives a dictionary object; .values() lists the DataArrays

# loop over each DataArray object
for data_arr in data_vars:
    
    # compute the regional weighted means; use only land cells for mean with regions with both ocean and land
    w_means = region_average(data_arr, regions, land_only = True)
    
    # must loop over each percentile with quantile DataArray
    if 'quantiles' in data_arr.name:
        for i in range(0, 4):
            
            # add a row to DataFrame object for each percentile
            row_name = f'{data_arr.name}_pct_{str(data_arr["quantile"][i].values)[2:]}'
            wm_df = wm_df.append(pd.Series(name = row_name, dtype = 'float64'))
            
            # populate the DataFrame object for each percentile
            for col_name in col_names:
                wm_df[col_name][row_name] = w_means[i, :].sel(abbrev = col_name)
    
    elif 'quantiles' not in data_arr.name:
        
        # add a row to DataFrame object
        row_name = data_arr.name
        wm_df = wm_df.append(pd.Series(name = row_name, dtype = 'float64'))
      
        # populate the DataFrame object
        for col_name in col_names:
            wm_df[col_name][row_name] = w_means.sel(abbrev = col_name)
            
# save DataFrame object using pickle; deconstructs and reconstucts data to save space
save_pw = '/home/ucfagtj/DATA/Dissertation/Data/Temperature/processed/ensemble_tas_variables_AR6.pickle'
with open(save_pw, 'wb') as f:
        pickle.dump(wm_df, f) 
print(f'File saved: {save_pw.split("/")[-1]}')

# close open Dataset and/or DataArray objects    
data.close()
del(data)

File saved: ensemble_tas_variables_AR6.pickle


### Table View of Weighted Regional Ensemble Means

In [42]:
print(ar6_df['land'][[40, 48, 56, 72]])

pre_ind_mean_ann_tas_abso     8.480703
current_mean_ann_tas_abso     9.172610
ssp126_mean_ann_tas_abso     11.011854
ssp370_mean_ann_tas_abso     13.455468
Name: land, dtype: float64


In [2]:
# load data weighted regional means data
wm_pw = f'/home/ucfagtj/DATA/Dissertation/Data/Temperature/processed/ensemble_tas_variables_AR6.pickle'
unpickle = open(wm_pw, 'rb')
wm_df = pickle.load(unpickle)

# create a DataFrame object with Regions and periods column
col_names = ['region', 'pre_ind', 'current', 'ssp126', 'ssp245', 'ssp370', 'ssp585', 
             'current_pct', 'ssp126_pct', 'ssp245_pct', 'ssp370_pct', 'ssp585_pct']
df = pd.DataFrame()
df = df.reindex(columns = col_names)

# define region names; excluding the two Antartica land regions
region_names = ['global', 'ocean', 'land', 'land_wo_antarctica'] + regionmask.defined_regions.ar6.land.abbrevs[: -2] 

# extract absolute and percentage increase for each region
for i, region in enumerate(region_names):
    
    # restrict weighted means DataFrame object to given region
    reg_data = wm_df[f'{region}']
    
    # extract absolute values
    pre_ind = reg_data['pre_ind_mean_ann_tas_abso']
    current = reg_data['current_mean_ann_tas_abso']
    ssp126 = reg_data['ssp126_mean_ann_tas_abso']
    ssp245 = reg_data['ssp245_mean_ann_tas_abso']
    ssp370 = reg_data['ssp370_mean_ann_tas_abso']
    ssp585 = reg_data['ssp585_mean_ann_tas_abso']
    
    # extract pre-industrial anomaly values; no percentage changes as meaningless due to different T scales
    current_pct = reg_data['current_mean_ann_tas_anom']
    ssp126_pct = reg_data['ssp126_mean_ann_tas_anom']
    ssp245_pct = reg_data['ssp245_mean_ann_tas_anom']
    ssp370_pct = reg_data['ssp370_mean_ann_tas_anom']
    ssp585_pct = reg_data['ssp585_mean_ann_tas_anom']

    # from Dictionary object holding a given region's data
    data = {'region': region, 'pre_ind': round(pre_ind, 2), 'current': round(current, 2),
            'current_pct': round(current_pct, 2), 'ssp126': round(ssp126, 2), 
            'ssp126_pct': round(ssp126_pct, 2), 'ssp245': round(ssp245, 2),
            'ssp245_pct': round(ssp245_pct, 2), 'ssp370': round(ssp370, 2),
            'ssp370_pct': round(ssp370_pct, 2), 'ssp585': round(ssp585, 2),
            'ssp585_pct': round(ssp585_pct, 2)}
    
    # add data as new entry to DataFrame object
    df = df.append(data, ignore_index = True, sort = False)

print(df['ssp126_pct'][4:].nsmallest(3), df['ssp585_pct'][4:].nlargest(3))
df

19    1.39
47    1.55
46    1.69
Name: ssp126_pct, dtype: float64 32    9.51
6     8.88
5     7.99
Name: ssp585_pct, dtype: float64


,region,pre_ind,current,ssp126,ssp245,ssp370,ssp585,current_pct,ssp126_pct,ssp245_pct,ssp370_pct,ssp585_pct
0,global,13.74,14.30,15.66,16.52,17.47,18.16,0.56,1.92,2.78,3.74,4.42
1,ocean,15.87,16.38,17.54,18.29,19.11,19.68,0.51,1.67,2.42,3.24,3.81
2,land,8.48,9.17,11.01,12.16,13.46,14.42,0.69,2.53,3.68,4.97,5.93
3,land_wo_antarctica,12.51,13.19,15.07,16.24,17.55,18.52,0.69,2.57,3.73,5.04,6.02
4,GIC,-19.78,-18.75,-16.55,-15.17,-13.76,-12.80,1.03,3.22,4.61,6.02,6.98
5,NWN,-5.03,-4.13,-1.43,-0.01,1.63,2.96,0.90,3.60,5.01,6.65,7.99
6,NEN,-7.18,-6.26,-3.23,-1.62,0.24,1.70,0.92,3.95,5.56,7.42,8.88
7,WNA,7.58,8.05,10.10,11.28,12.51,13.51,0.46,2.52,3.70,4.92,5.92
8,CNA,11.66,11.96,14.16,15.42,16.69,17.73,0.30,2.49,3.76,5.03,6.07
9,ENA,9.41,9.83,12.02,13.22,14.53,15.51,0.42,2.62,3.82,5.12,6.10


In [29]:
# calculate percentage increases from the pre-industrial
#current_pct = (reg_data['current_mean_ann_tas_anom'] / abs(pre_ind)) * 100
#ssp126_pct = (reg_data['ssp126_mean_ann_tas_anom'] / abs(pre_ind)) * 100
#ssp245_pct = (reg_data['ssp245_mean_ann_tas_anom'] / abs(pre_ind)) * 100
#ssp370_pct = (reg_data['ssp370_mean_ann_tas_anom'] / abs(pre_ind)) * 100
#ssp585_pct = (reg_data['ssp585_mean_ann_tas_anom'] / abs(pre_ind)) * 100
    
regions = regionmask.defined_regions.ar6.land
regions['RAR']

Region: Russian-Arctic (RAR / 28)
center: [109.22387611  71.58559615]